In [21]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

import pynbody
from pynbody.analysis import profile

import os
import glob

from tqdm.notebook import tqdm

import sys
sys.path.append('../../py/')

from cosmoSim import cosmoSim

In [22]:
matplotlib.rc('xtick', labelsize=12)
matplotlib.rc('ytick', labelsize=12)
matplotlib.rcParams['font.size']=12

In [23]:
data_path = "/home/ryan/Data/"
base_path="../../data_prods/"

out_path = '../../plots/movie_710_DM/'


try:
    os.mkdir(out_path)
except:
    print(f'{out_path} already exists!')

../../plots/movie_710_DM/ already exists!


In [24]:
CDM_RUNS = [
    'run_CDM_710_DM'
]

tcDM_search = '../../data_prods/run_2cDM_710_DM_power00*Vkick*'

tcDM_RUNS = [ os.path.basename(x) for x in sorted(glob.glob(tcDM_search)) ]

In [25]:
z_max = 5

with open('../../file_parts/ExpansionList_128', 'r') as f:
    lines = f.read().split('\n')

a = np.array([ float(l.strip().split(' ')[0]) for l in lines if l])

zs = 1/a - 1

zs = zs[ zs < z_max ]

In [26]:
Vkicks = [ cosmoSim(x).Vkick for x in tcDM_RUNS ]

c_norm = matplotlib.colors.LogNorm(vmin=np.amin(Vkicks), vmax=np.amax(Vkicks))
c_map = matplotlib.cm.plasma

s_map = matplotlib.cm.ScalarMappable(cmap=c_map, norm=c_norm)

In [27]:
soft_eff = 100000 / 2**10 / 39

In [ ]:
def calculate_profile(run_name, redshift):
    
    run = cosmoSim(run_name, base_path=base_path)
    idx = run.redshift_to_index(redshift)
    # load the file
    # print(data_path + f'/{run_name}/snap_{idx:03}.hdf5')
    f = pynbody.load(data_path + f'/{run_name}/snap_{idx:03}.hdf5')
    h = f.halos(subhalos=True)

    f.physical_units()
    
    # only care about most massive halo for now
    pynbody.analysis.faceon(h[0])
    p_3d = profile.Profile(f, rmin = '.05 kpc', rmax = '500 kpc', ndim = 3)
    return p_3d['rbins'].in_units('kpc'), p_3d['density'].in_units('Msol kpc^-3')

In [30]:
for redshift in tqdm(zs):

    fig, ax = plt.subplots(1, 1, figsize=[12, 12])

    runf = cosmoSim(CDM_RUNS[0])
    idx = runf.redshift_to_index(redshift)

    # diagnostic only
    # if idx < 127:
    #     plt.close(fig)
    #     continue

    # plot 2cDM runs
    for rn in tcDM_RUNS:
        run = cosmoSim(rn)

        rbins, masses = calculate_profile(rn, redshift)

        plt.plot(rbins, masses, color=s_map.to_rgba(run.Vkick))

    fig.colorbar(s_map, label='$V_{kick}$ [km s$^{-1}$]', ax=ax)

    # plot CDM run
    for rn in CDM_RUNS:

        rbins, masses = calculate_profile(rn, redshift)

        plt.plot(rbins, masses, color='k', linestyle=(5, (10, 3)))

    # plot formatting
    ax.set_xlabel('$r$ [kpc]')
    ax.set_ylabel(r'$\rho$ [M$_{\odot}$ kpc$^{-3}$]')

    ax.vlines(soft_eff, 10**2, 10**50, colors='k', linestyles='dashed')
    ax.vlines(2.8*soft_eff, 10**2, 10**50, colors='k', linestyles='dashdot')

    ax.set_ylim([10**3, 10**10])

    ax.plot([],[], label=f'z = {redshift:.3f}', alpha=0)

    ax.set_yscale("log")
    ax.set_xscale("log")
    ax.grid(True, which="both", ls="-")
    ax.set_box_aspect(aspect=1)
    ax.legend()
    plt.title('Density Profile 710 DM')

    fname = f'density_710_DM_{idx:03}.png'
    plt.savefig(out_path + fname)

    plt.close(fig)

  0%|          | 0/83 [00:00<?, ?it/s]